In [1]:
%matplotlib inline
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
from scipy.spatial import distance

In [19]:
import os

cnt = 0
stats = {}
for root, dirs, files in os.walk('../artifacts'):
    for name in files:
        path = root.split('/') 
        if len(path) == 4:
            algo, map_type = path[-2:]
            if algo not in stats:
                stats[algo] = {}
            if map_type not in stats[algo]:
                stats[algo][map_type] = np.array([])
            with open(os.path.join(root, name)) as f:
                for i, line in enumerate(f.readlines()[::2]):
                    path = list(map(int, line.strip().split()))
                    xs, ys = path[::2], path[1::2]
                    path_len = np.sum([distance.euclidean([x, y], 
                                                         [xs[i + 1], ys[i + 1]]) 
                                      for i, (x, y) in enumerate(list(zip(xs, ys))[:-1])])
                    stats[algo][map_type] = np.append(stats[algo][map_type], path_len)
                    if algo == 'anya' and map_type =='maze':
                        if cnt == 1881:
                            print(path_len)
                            print(path)
                            print(root, name, i)
                        cnt += 1

22.486832980505138
[271, 452, 271, 440, 270, 440, 267, 449]
../artifacts/anya/maze maze512-4-9.map.scen 61


In [102]:
algos = ['anya', 'astar.eucidian', 'astar.diagonal']
maps = ['baldurs-gate', 'maze', 'my-life-for-the-horde']


for m in maps:
    theta, anya = stats['2k_3.euclidian'][m], stats['anya'][m]
    print(m, 100 * (np.mean(np.divide(theta, anya, out=np.ones(anya.shape[0]), where=anya!=0)) - 1))

baldurs-gate 4.222917556225303
maze 4.141839840448358
my-life-for-the-horde 4.974658359605777


In [15]:
np.argmax(stats['anya']['maze'] - stats['2k_3.euclidian']['maze'])

1881

In [17]:
stats['anya']['maze'][1881]
# stats['astar.diagonal']['my-life-for-the-horde'][732]

22.486832980505138

In [105]:
for k in maze:
    print(k)

theta.euclidian
astar.chebyshev
anya
theta.diagonal
astar.manhattan
theta.trivial
astar.diagonal
theta.manhattan
astar.euclidian
theta.chebyshev
astar.trivial
